In [41]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [42]:
pip install pandas==1.1.0

     |████████████████████████████████| 10.4 MB 848 kB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.2
    Uninstalling pandas-1.1.2:
      Successfully uninstalled pandas-1.1.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
cd '/content/gdrive/My Drive/master/engine'

# Import Libraries

In [1]:
# General imports
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import math
import random
import sys, gc, time
import os

# data
import datetime
import itertools
import json
import pickle

# visualize
import seaborn as sns
import matplotlib.pyplot as plt

# model
import lightgbm as lgb
from lightgbm import LGBMRegressor

# custom modules
# from engine.features_yj import Features
from preprocess import load_df_added, drop_useless, check_na, run_label_all, remove_outliers, run_stdscale

In [2]:
lag_col1 = ['lag_scode_count','lag_mcode_price','lag_mcode_count','lag_bigcat_price','lag_bigcat_count',
            'lag_bigcat_price_day','lag_bigcat_count_day','lag_small_c_price','lag_small_c_count']

lag_col2 = ['rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21', 'rolling_mean_28','mean_sales_origin',
            'lag_sales_wd_1', 'lag_sales_wd_2','lag_sales_wd_3','lag_sales_wd_4', 'lag_sales_wd_5', 
            'lag_sales_wk_1','lag_sales_wk_2', 'ts_pred']

cat_col = ['상품군','weekdays','show_id','small_c','middle_c','big_c','original_c',
                        'pay','months','hours_inweek','weekends','japp','parttime',
                        'min_start','primetime','prime_origin','prime_smallc',
                        'freq','bpower','steady','men','luxury',
                        'spring','summer','fall','winter','rain']

# Preprocessing functions

In [3]:
## simple function that will be used for run_preprocess
def drop_useless(df, keepshowid = True):
    """
    :objective: drop useless features for model. save 'show_id' just in case
    :return: pandas dataframe
    """
    #useless features
    xcol = ['방송일시', '노출(분)', '마더코드', '상품명', 'exposed', 'ymd', 'volume',
            'years','days','hours','week_num','holidays', 'red', 'min_range','brand',
            'small_c_code','middle_c_code','big_c_code','sales_power']
    col = [x for x in df.columns if x in xcol]
    df = df.drop(columns = col)
    if keepshowid:
        df = df.copy()
    return df
    
def na_to_zeroes(df):
    """
    :objective: Change all na's to zero.(just for original lag!)
    :return: pandas dataframe
    """
    xcol = [x for x in df.columns if x in lag_col1+lag_col2]
    for col in xcol:
        df[col] = df[col].fillna(0)

    return df

## run preprocessing in a shot
## pca is optional and only applied to numeric features other than 'lag'
## NOTICE: removing outliers were run prior to dividing train/val
## if replace = True, new PCA will replace corresponding numerical columns
## if you want to simply add PCA columns to original data, set replace = False
def run_preprocess(df, pca = True, replace = True):
    """
    :objective: Run Feature deletion, NA imputation, label encoding, pca(optional)
    :return: pandas dataframe
    """
    df = drop_useless(df)
    df = na_to_zeroes(df)
    # df = remove_outliers(df)
    df = run_label_all(df)
    df1 = df.copy()
    return df1


# Criteria function

In [4]:
def get_mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def get_rmse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    return rmse

# Function needed for train

In [5]:
## Seeder
def seed_everything(seed=127):
    random.seed(seed)
    np.random.seed(seed)

## CV splits
def cv_split(df, month, printprop = False):
    split = int(df[df['months']==month].index.values.max())
    prop = str(split/df.shape[0])
    if printprop:
        print(f'Proportion of train set is {prop}')
        return split
    else:
        return split
        
## Divide into train/test
def divide_train_val(df_pp, month, drop):
    split = cv_split(df = df_pp, month = month)
    train_x = df_pp.iloc[:split].drop(['index','취급액']+drop, axis = 1)
    train_y = df_pp.iloc[:split,:].취급액
    val_x = df_pp.iloc[split:,:].drop(columns = ['index','취급액']+drop, axis = 1)
    val_y = df_pp.iloc[split:,:].취급액
    return train_x, train_y, val_x, val_y

# Train with full data(wd+wk)

In [190]:
df_full_lag = pd.read_pickle("../data/20/train_fin_light_ver.pkl")

In [193]:
hung = pd.read_pickle("../data/20/hung_firstwk_featured.pkl")
hung_PP = run_preprocess(hung, pca = False, replace =False)
hung_cols = hung_PP.columns.to_list()
hung_times = hung.iloc[:125]['방송일시'] # for output

In [194]:
df_full_lag = df_full_lag[hung_cols+['취급액']]

In [195]:
df_full_lag = run_preprocess(df_full_lag, pca = False, replace =False)
df_full_lag.reset_index(inplace=True)
train_x, train_y, val_x, val_y = divide_train_val(df_full_lag, 8, drop = ['small_c','original_c','상품코드'])

In [196]:
print(len(train_x.columns))
print(len(hung_cols)) #small_c, original_c,'상품코드'

56
59


In [197]:
params = {
    'feature_fraction': 1,
    'learning_rate': 0.001,
    'min_data_in_leaf': 135,
    'n_estimators': 3527,
    'num_iterations': 2940,
    'subsample': 1,
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'mape',
    'categorical_feature': [3, 9, 10, 11]
}
gbm = LGBMRegressor(**params)


def run_lgbm(train_x, train_y, val_x, val_y):
    seed_everything(seed=127)
    estimator = gbm.fit(train_x,train_y,
                      eval_set=[(val_x, val_y)],
                      verbose = 100,
                      eval_metric = 'mape',
                      early_stopping_rounds = 100
                      )
    lgbm_preds = gbm.predict(val_x, num_iteration= estimator.best_iteration_)
    lgbm_preds[lgbm_preds < 0] = 0

    # plot
    x = range(0,val_y.shape[0])
    plt.figure(figsize=(50,10))
    plt.plot(x,val_y,label='true')
    plt.plot(x,lgbm_preds, label='predicted')
    plt.legend()

    plt.show()

    # show scores
    print(f'MAPE of best iter is {get_mape(val_y,lgbm_preds)}')
    print(f'RMSE of best iter is {get_rmse(val_y,lgbm_preds)}')

    # save model
    data_type = 'all_lag'
    model_name = '../data/saved_models/'+'lgbm_opt_mape_lr001_'+data_type+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))


In [ ]:
run_lgbm(train_x, train_y, val_x, val_y)

# Prediction for Hungarian Optimization

In [198]:
lgbm_model_path = "../data/saved_models/lgbm_opt_mape_lr001_all_lag.bin"
estimator = pickle.load(open(lgbm_model_path, 'rb'))
lgbm_preds_opt = estimator.predict(hung_PP.drop(columns=['small_c','show_id','상품코드']))

In [224]:
hung_PP.head()

,상품코드,상품군,판매단가,original_c,small_c,middle_c,big_c,months,weekdays,hours_inweek,...,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28,mean_sales_origin,lag_sales_1,lag_sales_2,lag_sales_5,lag_sales_7,ts_pred
0,0,3,40900.0,6,50,3,4,6,1,6,...,4.602531e+07,4.555105e+07,4.449858e+07,4.341127e+07,4.385107e+07,52283000.0,55365000.0,47220000.0,38682500.0,1.020988e+08
1,0,3,40900.0,6,50,3,4,6,1,7,...,4.602531e+07,4.555105e+07,4.449858e+07,4.341127e+07,4.385107e+07,105205500.0,101303000.0,65229000.0,61955000.0,1.219246e+08
2,0,3,40900.0,6,50,3,4,6,1,8,...,4.602531e+07,4.555105e+07,4.449858e+07,4.341127e+07,4.385107e+07,179340500.0,205426000.0,78794000.0,92893500.0,1.446663e+08
3,0,3,40900.0,6,50,3,4,6,1,9,...,4.602531e+07,4.555105e+07,4.449858e+07,4.341127e+07,4.385107e+07,154221000.0,175086000.0,107790500.0,115909500.0,1.631861e+08
4,0,3,40900.0,6,50,3,4,6,1,10,...,4.602531e+07,4.555105e+07,4.449858e+07,4.341127e+07,4.385107e+07,169177500.0,203866000.0,133019500.0,137447500.0,1.688854e+08


In [200]:
lgbm_preds_opt

array([23568423.50655246, 27694413.75683077, 30159739.197232  , ...,
       22734777.06128463, 22734777.06128463, 22734777.06128463])

In [201]:
len(lgbm_preds_opt)

15625

In [202]:
hung_mat = lgbm_preds_opt.reshape((125,125)) #rows: items  cols: time

In [203]:
hung_mat.shape

(125, 125)

In [204]:
hung_mat

array([[23568423.50655246, 27694413.75683077, 30159739.197232  , ...,
        30943734.63319469, 30943734.63319469, 30943734.63319469],
       [23516226.22800101, 23516226.22800101, 27642216.47827931, ...,
        29913225.62626702, 29913225.62626702, 30076353.44347798],
       [30076353.44347798, 30076353.44347798, 30076353.44347798, ...,
        30943734.63319469, 30943734.63319469, 30943734.63319469],
       ...,
       [ 9178779.81546641,  9178779.81546641,  9795149.85254762, ...,
         9863989.68948797,  9863989.68948797,  9863989.68948797],
       [ 9863989.68948797,  9863989.68948797,  9863989.68948797, ...,
         9868319.948137  ,  9868319.948137  ,  9868319.948137  ],
       [19701692.40482032, 20590786.29094524, 20601155.56281116, ...,
        22734777.06128463, 22734777.06128463, 22734777.06128463]])

# Hungarian package

In [205]:
from munkres import Munkres, print_matrix

In [206]:
matrix = hung_mat
cost_matrix = []
for row in matrix:
    cost_row = []
    for col in row:
        cost_row += [sys.maxsize - col]
    cost_matrix += [cost_row]

In [207]:
m = Munkres()
indexes = m.compute(cost_matrix)
# print_matrix(matrix, msg='Highest profit through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
#     print(f'({row}, {column}) -> {value}')

print(f'total profit={total}')

total profit=2650256010.332908


In [208]:
indexes #(row, col)

[(0, 108),
 (1, 65),
 (2, 82),
 (3, 66),
 (4, 79),
 (5, 23),
 (6, 2),
 (7, 121),
 (8, 70),
 (9, 1),
 (10, 0),
 (11, 68),
 (12, 51),
 (13, 78),
 (14, 87),
 (15, 71),
 (16, 67),
 (17, 77),
 (18, 110),
 (19, 89),
 (20, 91),
 (21, 88),
 (22, 7),
 (23, 8),
 (24, 41),
 (25, 27),
 (26, 40),
 (27, 28),
 (28, 16),
 (29, 6),
 (30, 119),
 (31, 3),
 (32, 92),
 (33, 61),
 (34, 60),
 (35, 98),
 (36, 55),
 (37, 59),
 (38, 94),
 (39, 54),
 (40, 9),
 (41, 80),
 (42, 56),
 (43, 32),
 (44, 50),
 (45, 112),
 (46, 90),
 (47, 57),
 (48, 5),
 (49, 118),
 (50, 49),
 (51, 42),
 (52, 13),
 (53, 107),
 (54, 10),
 (55, 12),
 (56, 83),
 (57, 104),
 (58, 103),
 (59, 99),
 (60, 76),
 (61, 19),
 (62, 17),
 (63, 75),
 (64, 115),
 (65, 74),
 (66, 73),
 (67, 101),
 (68, 14),
 (69, 105),
 (70, 11),
 (71, 86),
 (72, 62),
 (73, 38),
 (74, 111),
 (75, 20),
 (76, 97),
 (77, 124),
 (78, 4),
 (79, 69),
 (80, 18),
 (81, 22),
 (82, 58),
 (83, 122),
 (84, 117),
 (85, 72),
 (86, 63),
 (87, 26),
 (88, 25),
 (89, 24),
 (90, 30),
 (9

In [209]:
full_items_list = pd.read_excel("../data/20/tmp_hung_firstweek_items.xlsx")

In [210]:
full_items_list.columns

Index(['Unnamed: 0', '마더코드', '노출(분)', '상품코드', '상품명', '상품군', '판매단가',
       'original_c', 'small_c', 'small_c_code', 'middle_c', 'middle_c_code',
       'big_c', 'big_c_code'],
      dtype='object')

In [211]:
full_items_list.drop(columns = ['Unnamed: 0', 'small_c_code', 'middle_c_code','big_c_code'], inplace = True)

In [212]:
full_items_list

,마더코드,노출(분),상품코드,상품명,상품군,판매단가,original_c,small_c,middle_c,big_c
0,100650,20.0,201971,잭필드 남성 반팔셔츠 4종,의류,59800,셔츠,티셔츠,남성의류,패션의류
1,100362,20.0,201150,에이유플러스 슈퍼선스틱 1004(최저가),이미용,39900,선스틱,선스틱,선케어,화장품/미용
2,100537,20.0,201616,[기간]제주바다자연산돔39마리,농수축,39900,갈치,생선,수산,식품
3,100383,20.0,201250,한라궁 황칠 제주오메기떡 2종 40개 (호박오메기20봉+팥오메기20봉),농수축,29900,모시떡,떡,가공식품,식품
4,100150,20.0,200424,LG전자 통돌이 세탁기 TR14WK1(화이트),가전,499900,세탁기,세탁기,생활가전,디지털/가전
...,...,...,...,...,...,...,...,...,...,...
120,100097,20.0,200266,무이자 올리고 가스와이드그릴레인지 프리미엄형 +버팔로 캠핑쿨러백,주방,129000,그릴,가스레인지,주방가전,디지털/가전
121,100150,20.0,200426,LG전자 통돌이 세탁기 TR14WK1(화이트),가전,499900,세탁기,세탁기,생활가전,디지털/가전
122,100155,20.0,200486,무이자 LG전자 매직스페이스 냉장고,가전,1499000,냉장고,냉장고,주방가전,디지털/가전
123,100290,20.0,202241,클라쎄 벽걸이 에어컨 MKRA06DTB,가전,449000,벽걸이에어컨,에어컨,계절가전,디지털/가전


In [213]:
hung_out = full_items_list.copy()
hung_out['방송일시'] = np.nan

In [214]:
for i in indexes:
    product = i[0]
    time = i[1]
    hung_out['방송일시'][product] = hung_times[time]

In [215]:
hung_out.drop(columns = ['노출(분)'], inplace = True)

In [216]:
hung_out.방송일시.isna().sum()

0

In [217]:
hung_out.sort_values(['방송일시'], ascending = True, inplace = True)

In [218]:
hung_out.set_index('방송일시', inplace = True)

In [219]:
hung_out.to_excel("../data/20/hung_output_tmp.xlsx")

In [67]:
# 6월 첫째주 예상 최대 매출 2,650,256,010원

In [220]:
hung_out

,마더코드,상품코드,상품명,상품군,판매단가,original_c,small_c,middle_c,big_c
방송일시,,,,,,,,,
2020-06-01 06:00:00,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,팬티,보정속옷,여성언더웨어/잠옷,패션의류
2020-06-01 07:00:00,100320,201016,올바로 문어발 샤워기 1+1,생활용품,54900,샤워기,샤워기/수전용품,욕실용품,생활/건강
2020-06-01 08:00:00,100501,201520,파격가 노비타 스마트 비데 무료설치(변기세정제),생활용품,189000,비데,비데/비데용품,욕실용품,생활/건강
2020-06-01 09:00:00,100728,202125,자연감성 풍기인견 여성란쥬세트,속옷,59900,이너웨어,보정속옷,여성언더웨어/잠옷,패션의류
2020-06-01 10:00:00,100639,201957,완도특大활전복 16~18미,농수축,59900,전복,해산물/어패류,수산,식품
...,...,...,...,...,...,...,...,...,...
2020-06-07 19:00:00,100203,200689,(무)[보루네오] 피올레 천연소가죽 소파 3인용,가구,749000,소파,소파,거실가구,가구/인테리어
2020-06-07 20:00:00,100148,200416,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 55형 55UN7850KNA,가전,1340000,TV,TV,영상가전,디지털/가전
2020-06-07 21:00:00,100428,201341,프로피쿡 스퀘어 에어프라이어 5.5L (종이호일),주방,89000,에어프라이어,에어프라이어,주방가전,디지털/가전


In [12]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import sys
import os

# data
import datetime
import json
import pickle

# visualize
import matplotlib.pyplot as plt

# model
import lightgbm as lgb
from lightgbm import LGBMRegressor

# from engine.preprocess import load_df, run_preprocess
from train import load_df, run_preprocess,divide_train_val, seed_everything, get_mape
from features import Features

# optimization
from munkres import Munkres


ModuleNotFoundError: No module named 'engine'

In [ ]:

###############################################################################
################################# Load Data ###################################
###############################################################################

## Set Directories
## Data is NOT RAW and has all features

local_DIR = '..'
MODELS_DIR = local_DIR + "/data/saved_models"
featured_DATA_DIR = local_DIR + '/data/fin_data'

## Import 2 types of dataset
## Descriptions:
#   hung1 : hungarian input for no hierarchical model
#   hung2 : hungarian input for hierarchical model

hung1 = load_df(featured_DATA_DIR + '/hung_featured_1.pkl')
hung1_PP = run_preprocess(hung1)
hung1_cols = hung1_PP.columns.to_list()  # check!
hung1_times = hung1_PP.iloc[:125]['방송일시']  # for output

hung2 = Features(types="hungarian")
hung2 = hung2.run_hungarian()
# item list for hung2
hung_list = hung2[['상품코드','상품명']].drop_duplicates()
hung_list['row_num'] = list(range(0,len(hung_list)))
hung2_PP = run_preprocess(hung2)
hung2_cols = hung2_PP.columns.to_list()
hung2_times = hung2_PP.iloc[:660]['방송일시']  # for output


In [8]:
!cd ../

In [ ]:
hung1_PP

In [5]:

####################################################################
########################### New train ##############################
####################################################################
"""
Adjust full preprocess train dataset for modeling
as we cannot define some time lag features for hungarian input.
Drop those columns and train 
"""
df_full_lag = pd.read_pickle(featured_DATA_DIR + "/train_fin_light_ver.pkl")
df_full_lag = df_full_lag[hung1_cols+['취급액']]
df_full_lag = run_preprocess(df_full_lag)
df_full_lag.reset_index(inplace=True)
train_x, train_y, val_x, val_y = divide_train_val(df_full_lag, 8, drop=['small_c', 'original_c', '상품코드'])

####################################################################
########################### Light GBM ##############################
####################################################################
params = {
    'feature_fraction': 1,
    'learning_rate': 0.001,
    'min_data_in_leaf': 135,
    'n_estimators': 3527,
    'num_iterations': 2940,
    'subsample': 1,
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'mape',
    'categorical_feature': [3, 9, 10, 11]
}
gbm = LGBMRegressor(**params)


def run_lgbm(train_x, train_y, val_x, val_y):
    seed_everything(seed=127)
    estimator = gbm.fit(train_x,train_y,
                      eval_set=[(val_x, val_y)],
                      verbose = 100,
                      eval_metric = 'mape',
                      early_stopping_rounds = 100
                      )
    lgbm_preds = gbm.predict(val_x, num_iteration= estimator.best_iteration_)
    lgbm_preds[lgbm_preds < 0] = 0

    # plot
    x = range(0,val_y.shape[0])
    plt.figure(figsize=(50,10))
    plt.plot(x,val_y,label='true')
    plt.plot(x,lgbm_preds, label='predicted')
    plt.legend()

    plt.show()

    # show scores
    print(f'MAPE of best iter is {get_mape(val_y,lgbm_preds)}')

    # save model
    data_type = 'all_lag'
    model_name = MODELS_DIR+'lgbm_opt_mape_lr001_'+data_type+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

run_lgbm(train_x, train_y, val_x, val_y)

####################################################################
########################### Predict ################################
####################################################################

lgbm_model_path = MODELS_DIR + 'lgbm_opt_mape_lr001_all_lag.bin'
estimator = pickle.load(open(lgbm_model_path, 'rb'))
lgbm_preds_opt1 = estimator.predict(hung1_PP.drop(columns=['small_c', 'show_id', '상품코드']))
lgbm_preds_opt2 = estimator.predict(hung2_PP.drop(columns=['small_c', 'show_id', '상품코드']))

####################################################################
########################### Hung1 ##################################
####################################################################

hung_mat = lgbm_preds_opt1.reshape((125, 125))  #rows: items, cols: time
matrix = hung_mat
cost_matrix = []
for row in matrix:
    cost_row = []
    for col in row:
        cost_row += [sys.maxsize - col]
    cost_matrix += [cost_row]

m = Munkres()
indexes = m.compute(cost_matrix)
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value

print(f'total profit={total}')

full_items_list = pd.read_excel("../data/20/tmp_hung_firstweek_items.xlsx")
full_items_list.drop(columns=['Unnamed: 0', 'small_c_code', 'middle_c_code','big_c_code'], inplace=True)
hung_out = full_items_list.copy()
hung_out['방송일시'] = np.nan
for i in indexes:
    product = i[0]
    time = i[1]
    hung_out['방송일시'][product] = hung1_times[time]
hung_out.drop(columns=['노출(분)'], inplace=True)
hung_out.sort_values(['방송일시'], ascending=True, inplace=True)
hung_out.set_index('방송일시', inplace=True)
hung_out.to_excel("../data/20/hung1_output.xlsx")

KeyError: "['index'] not in index"